In [1]:
from tokenizer import *

In [2]:
path = "./datasets/midi_songs/*.mid"
midi_list = load_midi_folder(path)
sequences = midi2seq(midi_list)

[INFO] Load successfully 13 file midi from source
[INFO] Convert midi to sequences successfully
	[+] Note: 1108
	[+] Symbol: 104
	[+] Track: 348


In [3]:
midi_list[0].key_signature_changes[0]

KeySignature(key_name=F, key_number=5, time=0)

In [4]:
def scale_time(time_sequence: list) -> list:
    new_seq = []
    back_note = ""
    back_symbol = ""
    for word in time_sequence:
        val = int(word.split("_")[-1])
        if val == 16:
            if "note_" in word and back_note == "":
                new_seq.append("note_8")
                back_note = word 
            if "note_" in word and back_note == word:
                back_note = ""
                continue
            if "symbol_" in word and back_symbol == "":
                new_seq.append("symbol_8")
                back_symbol = word
            if "symbol_" in word and back_symbol == word:
                back_symbol = ""
                continue
        else:
            new_seq.append(word)
    return new_seq

In [5]:
def seq2seq_idx(sequences: list) -> list:
    time_sequences = []
    note_sequences = []
    
    # Split into time frame and note frame
    for midi_seq in sequences:
        note_seq = list(filter(lambda word: "note" in word, midi_seq))
        note_seq = list(map(lambda word: f"note_{word.split('_')[1]}", note_seq))
        time_seq = list(map(lambda word: f"{word.split('_')[0]}_{word.split('_')[-1]}" , midi_seq))
        time_seq = scale_time(time_seq)
        time_seq = list(map(lambda word: VOCABULARY["TIME"].index(word), time_seq))
        note_sequences.append(note_seq)
        time_sequences.append(time_seq)
    
    # Convert note frame to tone C
    for idx_seq, note_seq in enumerate(note_sequences):
        scales = [int(word.split("_")[-1])//12 for word in note_seq]
        min_scales = min(scales)*12
        note_seq = list(map(lambda word: f"note_{int(word.split('_')[-1])-min_scales-(int(word.split('_')[-1])%12==1 or int(word.split('_')[-1])%12==6)}", note_seq))
        note_seq = list(map(lambda word: VOCABULARY["NOTE"].index(word), note_seq))
        note_sequences[idx_seq] = note_seq

    return note_sequences, time_sequences

In [6]:
note_sequences, time_sequences = seq2seq_idx(sequences)

In [7]:
note_tensor = note2tensor(note_sequences, 16, 2)
time_tensor = time2tensor(time_sequences, 8, 2)

In [23]:
import functools
concat = functools.reduce(lambda x1, x2: x1+x2, note_sequences)

In [24]:
concat = [VOCABULARY["NOTE"][int(i)].split("_")[-1] for i in concat]

In [26]:
from collections import Counter
ct = Counter(concat)
ct

Counter({'19': 56,
         '14': 187,
         '21': 21,
         '16': 127,
         '12': 201,
         '7': 183,
         '24': 22,
         '26': 18,
         '28': 14,
         '31': 1,
         '11': 31,
         '9': 125,
         '4': 43,
         '2': 49,
         '17': 1,
         '5': 17,
         '0': 12})

In [13]:
note_tensor.shape

torch.Size([66, 18])

In [8]:
torch.save(note_tensor, "./data_gen/note_tensor_v2.pt")
torch.save(time_tensor, "./data_gen/time_tensor_v2.pt")

In [1]:
from generate import *

In [2]:
model_note = load_model("./checkpoints/model_note_frames_v2.pt")
model_time = load_model("./checkpoints/model_time_frames_v2.pt")
token = [1]

In [69]:
seg_note = generate(model=model_note, x=token, min_length=15, max_length=128)
seg_time = generate(model=model_time, x=token, min_length=85, max_length=128)
time_frame = process_timeframe(seg_time)
note_frame = process_noteframe(seg_note)
print(time_frame)
print(note_frame)


['note_4', 'symbol_2']
[4, 7, 29, 11, 0, 4, 4, 17, 28, 17, 24, 5, 31, 19, 16, 7, 26, 7, 14, 14, 4, 2, 7, 28, 24, 12, 16, 21, 26, 9, 26, 0, 17, 12, 5, 23, 2, 7, 7, 2, 17, 5, 31]


In [61]:

# torch.save(time_frame, "./data_gen/pred_time_frame_v2.pt")
# torch.save(note_frame, "./data_gen/pred_note_frame_v2.pt")

midi_obj = frame2midi(note_frame, time_frame,save_path="./midi_gen/b1_2_4.mid", numerator=2)

[+] Save midi file sucessfully! File name: ./midi_gen/b1_2_4.mid
